In [115]:
import pandas as pd
from deep_translator import GoogleTranslator

In [116]:
df = pd.read_csv("livros.csv")


In [117]:
df.shape #linhas e colunas

(174, 6)

In [118]:
df.head()

,Book,Author(s),Original language,First published,Approximate sales in millions,Genre
0,A Tale of Two Cities,Charles Dickens,English,1859,200.0,Historical fiction
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200.0,Novella
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy
3,And Then There Were None,Agatha Christie,English,1939,100.0,Mystery
4,Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100.0,Family saga


In [119]:
df.info() #tipos de cada coluna

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Book                           174 non-null    object 
 1   Author(s)                      174 non-null    object 
 2   Original language              174 non-null    object 
 3   First published                174 non-null    int64  
 4   Approximate sales in millions  174 non-null    float64
 5   Genre                          118 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 8.3+ KB


In [120]:
traducao_colunas = {
    'Book':'livro', 
    'Author(s)':'Autor', 
    'Original language':'Idioma', 
    'First published': 'Ano',
    'Approximate sales in millions':'Vendas', 
    'Genre':'Gênero'
}

In [121]:
df.rename(columns=traducao_colunas, inplace=True)
df.columns

Index(['livro', 'Autor', 'Idioma', 'Ano', 'Vendas', 'Gênero'], dtype='object')

In [122]:
df.isnull().sum()
#problema com a coluna gênero

livro      0
Autor      0
Idioma     0
Ano        0
Vendas     0
Gênero    56
dtype: int64

In [123]:
df['Autor'].value_counts().head(10) #conta e printa os 10 maiores valores

Autor
J. K. Rowling       7
Dan Brown           3
Suzanne Collins     3
Roald Dahl          3
George Orwell       2
Ken Follett         2
E. L. James         2
Albert Camus        2
Erskine Caldwell    2
Cao Xueqin          1
Name: count, dtype: int64

In [124]:
autores_unicos = pd.DataFrame(df['Autor'].unique(), columns=['Autor'])
autores_unicos.head()

,Autor
0,Charles Dickens
1,Antoine de Saint-Exupéry
2,J. K. Rowling
3,Agatha Christie
4,Cao Xueqin


In [125]:
autores_unicos['Autor_id'] = autores_unicos.index + 1
autores_unicos.head()

,Autor,Autor_id
0,Charles Dickens,1
1,Antoine de Saint-Exupéry,2
2,J. K. Rowling,3
3,Agatha Christie,4
4,Cao Xueqin,5


In [126]:
df = df.merge(autores_unicos, on='Autor', how='left')

In [127]:
autores = pd.DataFrame(df['Autor'].unique(), columns=['nome'])
autores.head()

,nome
0,Charles Dickens
1,Antoine de Saint-Exupéry
2,J. K. Rowling
3,Agatha Christie
4,Cao Xueqin


In [128]:
with open('autores.sql', 'w', encoding='utf-8') as arquivo:
    for _, row in autores.iterrows():
        nome = row['nome'].replace("'", "''")
        arquivo.write(f"INSERT INTO autores (nome) VALUES ('{nome}');\n")

In [129]:
df['Gênero'].isnull().sum()

np.int64(56)

In [130]:
df['Gênero'] = df['Gênero'].fillna('Unknown')
df.tail(10)

,livro,Autor,Idioma,Ano,Vendas,Gênero,Autor_id
164,The Joy of Sex,Alex Comfort,English,1972,10.0,Unknown,148
165,The Gospel According to Peanuts,Robert L. Short,English,1965,10.0,Unknown,149
166,The Subtle Art of Not Giving a Fuck,Mark Manson,English,2016,10.0,Unknown,150
167,Life of Pi,Yann Martel,English,2001,10.0,Unknown,151
168,The Front Runner,Patricia Nell Warren,English,1974,10.0,Unknown,152
169,The Goal,Eliyahu M. Goldratt,English,1984,10.0,Unknown,153
170,Fahrenheit 451,Ray Bradbury,English,1953,10.0,Unknown,154
171,Angela's Ashes,Frank McCourt,English,1996,10.0,Unknown,155
172,The Story of My Experiments with Truth (સત્યના...,Mohandas Karamchand Gandhi,Gujarati,1929,10.0,Unknown,156
173,Bridget Jones's Diary,Helen Fielding,English,1996,10.0,Unknown,157


In [131]:
generos = pd.DataFrame(df['Gênero'].unique(), columns=['genero'])
generos.head(100)

,genero
0,Historical fiction
1,Novella
2,Fantasy
3,Mystery
4,Family saga
...,...
76,"Historical fiction, war novel"
77,Erotica
78,Science fiction
79,Dystopian fiction


In [132]:
#traduzindo os dados da coluna gênero do DF
generos['Gênero_pt'] = generos['genero'].apply(
    lambda x: GoogleTranslator(source='auto', target='pt').translate(x)
)

In [133]:
generos.head(85)

,genero,Gênero_pt
0,Historical fiction,Ficção histórica
1,Novella,Novella
2,Fantasy,Fantasia
3,Mystery,Mistério
4,Family saga,Saga da família
...,...,...
76,"Historical fiction, war novel","Ficção histórica, romance de guerra"
77,Erotica,Erótica
78,Science fiction,ficção científica
79,Dystopian fiction,Ficção distópica


In [134]:
generos['Gênero_pt'] = generos['Gênero_pt'].replace('Novella', 'Novela')
generos['Gênero_pt'] = generos['Gênero_pt'].replace('Saga da família', 'Saga familiar')
generos.head()

,genero,Gênero_pt
0,Historical fiction,Ficção histórica
1,Novella,Novela
2,Fantasy,Fantasia
3,Mystery,Mistério
4,Family saga,Saga familiar


In [135]:
generos['Gênero_pt'] = generos['Gênero_pt'].str.lower()
generos.head()

,genero,Gênero_pt
0,Historical fiction,ficção histórica
1,Novella,novela
2,Fantasy,fantasia
3,Mystery,mistério
4,Family saga,saga familiar


KeyError: 'Gênero'

In [ ]:
#Criando os scripts para inserção de dados no sql

with open('generos.sql', 'w', encoding='utf-8') as arquivo:
    for nome in generos['Gênero_pt']:
        nome_validado = nome.replace("'", "''")
        arquivo.write(f"INSERT INTO generos (nome) VALUES ('{nome_validado}');\n")

